# Step 5 Portfolio Optimization (VBTLX & VFIAX)

This notebook follows the Step 5 instructions using the provided monthly returns for VBTLX/VFIAX and AAPL monthly prices.
It uses only the Python standard library so it can run without extra installs.


In [ ]:
import csv
import zipfile
from datetime import datetime, timedelta
from math import sqrt
from pathlib import Path
from xml.etree import ElementTree as ET

XLSX_PATH = Path('_b01664a045fe7720c1bf143fe3fef7a9_VBTLX-and-VFIAX-Monthly-Returns.xlsx')
AAPL_MONTHLY_PATH = Path('Historical Stock Data') / '_58217c10deb55d59805ab8050cd390de_aapl_monthly-_2_.csv'

if not XLSX_PATH.exists():
    raise FileNotFoundError(XLSX_PATH)
if not AAPL_MONTHLY_PATH.exists():
    raise FileNotFoundError(AAPL_MONTHLY_PATH)


In [ ]:
def excel_date_to_date(serial):
    base = datetime(1899, 12, 30)
    return (base + timedelta(days=serial)).date()

def load_xlsx_rows(path, sheet_name='xl/worksheets/sheet1.xml'):
    with zipfile.ZipFile(path) as zf:
        sheet_xml = zf.read(sheet_name)
        shared_xml = zf.read('xl/sharedStrings.xml')

    ns = {'m': 'http://schemas.openxmlformats.org/spreadsheetml/2006/main'}
    shared_root = ET.fromstring(shared_xml)
    shared = []
    for si in shared_root.findall('m:si', ns):
        texts = [t.text or '' for t in si.findall('.//m:t', ns)]
        shared.append(''.join(texts))

    sheet_root = ET.fromstring(sheet_xml)
    rows = {}
    for cell in sheet_root.findall('.//m:c', ns):
        ref = cell.attrib.get('r')
        if not ref:
            continue
        col = ''.join(ch for ch in ref if ch.isalpha())
        row = int(''.join(ch for ch in ref if ch.isdigit()))
        v = cell.find('m:v', ns)
        if v is None:
            continue
        value = v.text
        if cell.attrib.get('t') == 's':
            value = shared[int(value)]
        rows.setdefault(row, {})[col] = value
    return rows

def load_vanguard_returns(path):
    rows = load_xlsx_rows(path)
    records = []
    for row_num in sorted(rows):
        row = rows[row_num]
        if row.get('A') == 'Date' or 'A' not in row:
            continue
        try:
            date_serial = float(row['A'])
        except ValueError:
            continue
        date = excel_date_to_date(int(date_serial))
        vbtlx = float(row.get('B', 'nan'))
        vfiax = float(row.get('C', 'nan'))
        if vbtlx != vbtlx or vfiax != vfiax:
            continue
        records.append({'date': date, 'VBTLX': vbtlx, 'VFIAX': vfiax})
    records.sort(key=lambda item: item['date'])
    return records

def parse_aapl_monthly(path):
    def parse_date(value):
        return datetime.strptime(value, '%m/%d/%y').date()

    rows = []
    with path.open(newline='') as handle:
        reader = csv.DictReader(handle)
        for row in reader:
            rows.append({
                'date': parse_date(row['Date']),
                'adj_close': float(row['Adj Close']),
            })
    rows.sort(key=lambda item: item['date'])

    returns = []
    previous = None
    for row in rows:
        if previous is not None:
            ret = (row['adj_close'] - previous['adj_close']) / previous['adj_close'] * 100.0
            returns.append({'date': row['date'], 'ret': ret})
        previous = row
    return returns

def mean(values):
    return sum(values) / len(values)

def covariance(x, y):
    n = len(x)
    mx = mean(x)
    my = mean(y)
    return sum((xi - mx) * (yi - my) for xi, yi in zip(x, y)) / (n - 1)

def invert_2x2(a, b, c, d):
    det = a * d - b * c
    return d / det, -b / det, -c / det, a / det

def tangency_weights_two_assets(mu1, mu2, var1, var2, cov12):
    inv_a, inv_b, inv_c, inv_d = invert_2x2(var1, cov12, cov12, var2)
    raw_w1 = inv_a * mu1 + inv_b * mu2
    raw_w2 = inv_c * mu1 + inv_d * mu2
    total = raw_w1 + raw_w2
    return raw_w1 / total, raw_w2 / total


In [ ]:
records = load_vanguard_returns(XLSX_PATH)

# Use data through Dec 2015 for the optimal risky portfolio
train = [r for r in records if r['date'] <= datetime(2015, 12, 31).date()]

vb_returns = [r['VBTLX'] for r in train]
vf_returns = [r['VFIAX'] for r in train]

mu_vb = mean(vb_returns)
mu_vf = mean(vf_returns)
var_vb = covariance(vb_returns, vb_returns)
var_vf = covariance(vf_returns, vf_returns)
cov_vb_vf = covariance(vb_returns, vf_returns)

w_vb, w_vf = tangency_weights_two_assets(mu_vb, mu_vf, var_vb, var_vf, cov_vb_vf)
investment_vb = round(w_vb * 5_000_000)

def record_for_month(year, month):
    for row in records:
        if row['date'].year == year and row['date'].month == month:
            return row
    return None

def portfolio_return(row):
    return w_vb * row['VBTLX'] + w_vf * row['VFIAX']

jan_2016 = record_for_month(2016, 1)
jul_2016 = record_for_month(2016, 7)
jan_return = portfolio_return(jan_2016)
jul_return = portfolio_return(jul_2016)

portfolio_2016 = []
for row in records:
    if row['date'].year == 2016 and 1 <= row['date'].month <= 7:
        portfolio_2016.append({'date': row['date'], 'ret': portfolio_return(row)})

# AAPL monthly returns Jan-Jul 2016
aapl_returns = parse_aapl_monthly(AAPL_MONTHLY_PATH)
aapl_2016 = [r for r in aapl_returns if r['date'].year == 2016 and 1 <= r['date'].month <= 7]

def std(values):
    m = mean(values)
    return sqrt(sum((x - m) ** 2 for x in values) / (len(values) - 1))

std_aapl = std([r['ret'] for r in aapl_2016])
std_port = std([r['ret'] for r in portfolio_2016])
statement_1 = std_aapl > std_port

both_negative = any((a['ret'] < 0 and p['ret'] < 0) for a, p in zip(aapl_2016, portfolio_2016))
statement_2 = not both_negative

statement_3 = (
    max(r['ret'] for r in aapl_2016) > max(r['ret'] for r in portfolio_2016)
    and min(r['ret'] for r in portfolio_2016) < min(r['ret'] for r in aapl_2016)
)

jul_aapl = next(r for r in aapl_2016 if r['date'].month == 7)['ret']
jul_port = next(r for r in portfolio_2016 if r['date'].month == 7)['ret']
statement_4 = jul_aapl > jul_port

value = 5_000_000
for row in portfolio_2016:
    value *= (1 + row['ret'] / 100.0)

final_value = round(value)


In [ ]:
print('Weights (VBTLX, VFIAX): {:.2f}%, {:.2f}%'.format(w_vb * 100, w_vf * 100))
print('Investment in VBTLX ($):', investment_vb)
print('Jan 2016 portfolio return (%): {:.2f}'.format(jan_return))
print('Jul 2016 portfolio return (%): {:.2f}'.format(jul_return))
print('Statement 1:', statement_1)
print('Statement 2:', statement_2)
print('Statement 3:', statement_3)
print('Statement 4:', statement_4)
print('Final value end of Jul 2016 ($):', final_value)


## Final Answers

1. 3392295
2. -0.62%
3. 1.62%
4. The returns of AAPL have more variance from zero than the returns for the weighted portfolio of VBTLX and VFIAX; The return of AAPL is greater in July 2016 than for the weighted portfolio of VBTLX and VFIAX.
5. 5338415
